Install the tweepy library

In [1]:
# !pip install tweepy
# !pip install mysqlclient

Import required libraries

In [2]:
import os
import re
import time

import tweepy as tw
import pandas as pd
from datetime import datetime, timedelta, timezone
import MySQLdb


# For sentiment analysis of tweets
from textblob import TextBlob

Create Tweeter Consumer Keys and Access Tokens 

In [3]:
consumer_key= 'NFIO7bIgwQpdCZ7p7rgewKXss'
consumer_secret= '2QegNrIpcGCW6nCWFTL5JEsrDCmYZpQHJ8SdIT7V0ql5cQgFwV'
access_token= '3049227557-36ln3qak9xbxHvgNos45cf3uScGxuUdaDAR4vqs'
access_token_secret= 'pZ4Y0p31graKrAYAy41mYztvIrYFzof7awiTDWQr2ui9u'

Initialize AuthHandler

In [4]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

Define search words and date

In [5]:
search_words = "#Covid"
date_since = "2018-11-16"

In [6]:
tweets = tw.Cursor(api.search, q=search_words, lang="en", since=date_since).items(5)

In [7]:
tweets

Iterate through tweets using for loop

In [8]:
from datetime import datetime, timedelta
from email.utils import parsedate_tz

s='Mon Dec 07 17:03:03 +0000 2020'
def to_datetime(datestring):
    time_tuple = parsedate_tz(datestring.strip())
    dt = datetime(*time_tuple[:6])
    proper_date=(dt - timedelta(seconds=time_tuple[-1]))
    return proper_date

print(to_datetime(s))

dt_string = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(dt_string)

2020-12-07 17:03:03
2020-12-26 18:01:29


Retrieve tweets as a collection of list

In [9]:
[tweet.text for tweet in tweets]

['Beware of scams for COVID #Vaccines or tests! #COVID vaccines are free for everyone, and COVID tests are free for M… https://t.co/RKtjIZmJuz',
 'RT @AllergyHealth: I just got my 1st dose of #Coronavirus #vaccine &amp; I am halfway to protected against severe #COVID-19 disease. I’m still…',
 '"Like many in the #restaurant #industry, Walter Green has had a tough year — but after getting diagnosed with… https://t.co/AQ37fMVduj',
 'RT @AgentP22: Anyone else looking forward to the Ian Blackford tweet demanding that #covid rule breaker Nicola Sturgeon resign?\n\n#ResignStu…',
 'RT @CAChirag: Income Tax E Filing portal not working - have to login 10 to 15 times to upload one ITR \n\nComplete waste of Man-hours during…']

Search Tweets without Retweets

In [10]:
new_search = search_words + " -filter:retweets"

In [11]:
new_search

'#Covid -filter:retweets'

In [12]:
tweets = tw.Cursor(api.search, q=new_search, lang="en",since=date_since).items(5)

In [13]:
[tweet.text for tweet in tweets]

['Beware of scams for COVID #Vaccines or tests! #COVID vaccines are free for everyone, and COVID tests are free for M… https://t.co/RKtjIZmJuz',
 '"Like many in the #restaurant #industry, Walter Green has had a tough year — but after getting diagnosed with… https://t.co/AQ37fMVduj',
 '#Covid #Vaccines #Likely #Effective #Against new #Strains, #Doctor says\nhttps://t.co/RciRkWN3wH https://t.co/R8IGdBr01z',
 'A tragic love story in the form of a picture book for adults. You have to read it!\nhttps://t.co/UaIiNxT54d\n#covid… https://t.co/hYmGYS0Nat',
 'It’s Been 48 Hours Since My COVID Vaccine. This is How I Feel: https://t.co/o6FaWjKHJl. #covid #covidvaccine']

Get username and location

In [14]:
tweets = tw.Cursor(api.search,q=new_search, lang="en", since=date_since).items(5)

In [15]:
users_locs = [[tweet.user.screen_name,tweet.user.location,tweet.text] for tweet in tweets]

In [16]:
users_locs

[['stopcancerfund',
  'Washington, D.C.',
  'Beware of scams for COVID #Vaccines or tests! #COVID vaccines are free for everyone, and COVID tests are free for M… https://t.co/RKtjIZmJuz'],
 ['ALCOSales',
  'Burr Ridge, IL',
  '"Like many in the #restaurant #industry, Walter Green has had a tough year — but after getting diagnosed with… https://t.co/AQ37fMVduj'],
 ['BloGoalcom',
  '',
  '#Covid #Vaccines #Likely #Effective #Against new #Strains, #Doctor says\nhttps://t.co/RciRkWN3wH https://t.co/R8IGdBr01z'],
 ['OrBragi',
  'Rustenburg, South Africa',
  'A tragic love story in the form of a picture book for adults. You have to read it!\nhttps://t.co/UaIiNxT54d\n#covid… https://t.co/hYmGYS0Nat'],
 ['DrJenCaudle',
  'Philadelphia, PA',
  'It’s Been 48 Hours Since My COVID Vaccine. This is How I Feel: https://t.co/o6FaWjKHJl. #covid #covidvaccine']]

Create a Pandas Dataframe From A List of Tweet Data

In [17]:
tweet_df = pd.DataFrame(data=users_locs, columns=['user', "location","tweet"])

In [18]:
tweet_df

,user,location,tweet
0,stopcancerfund,"Washington, D.C.",Beware of scams for COVID #Vaccines or tests! ...
1,ALCOSales,"Burr Ridge, IL","""Like many in the #restaurant #industry, Walte..."
2,BloGoalcom,,#Covid #Vaccines #Likely #Effective #Against n...
3,OrBragi,"Rustenburg, South Africa",A tragic love story in the form of a picture b...
4,DrJenCaudle,"Philadelphia, PA",It’s Been 48 Hours Since My COVID Vaccine. Thi...


## Working With The Twitter Streaming API

With Streaming data we need to do the following:
1. Create database connection
2. Create a persistent connection to the Twitter API, and read each connection incrementally.
3. Process tweets quickly, store them in the database, and don’t let your program get backed up.
4. Handle errors and other issues properly.

Connect to MySql Database

In [19]:
#                           (server,MySQL username,MySQL pass, Database name).
connection = MySQLdb.connect("localhost","root","root","social_analytic")
cur = connection.cursor()

StreamListener Class

In [20]:
from tweepy import Stream
from tweepy.streaming import StreamListener
import tweepy
import time
import json
import sys


search_val="covid"

class listener(StreamListener):

    def on_data(self, data):
       
        try:
#             tweet_count=0
            
            all_data = json.loads(data)
            
#             search_val="covid"
            tweet_id=all_data["id"]
            created_at=to_datetime(all_data["created_at"])
            created_at_date=created_at.date()
            created_at_hour=created_at.time().strftime("%H")
            created_at_min=created_at.time().strftime("%H:%M")
            text = all_data["text"]
            source = all_data["source"]
#             local_created_at=created_at.str.replace(tzinfo=timezone.utc).astimezone(tz=None)
            local_created_at=all_data["created_at"]
            
            in_reply_to_status_id=all_data['in_reply_to_status_id_str']
            in_reply_to_user_id=all_data['in_reply_to_user_id_str']
            in_reply_to_screen_name=all_data['in_reply_to_screen_name']
            username=all_data['user']['name']
            user_screen_name=all_data['user']['screen_name']
            user_location=all_data['user']['location']
            user_url=all_data['user']['url']
            user_description=all_data['user']['description']
            user_verified_str=all_data['user']['verified']
            user_verified=0
            if user_verified_str==True:
                user_verified=1
            user_followers_count=all_data['user']['followers_count']
            user_friends_count=all_data['user']['friends_count']
            user_listed_count=all_data['user']['listed_count']
            user_favourites_count=all_data['user']['favourites_count']
            user_created_at=to_datetime(all_data['user']['created_at'])
            user_created_at_date=user_created_at.date()
            user_id=all_data['user']['id']
            user_profile_image_url_https=all_data['user']['profile_image_url_https']
            
#             coordinates_lat=all_data['coordinates'][0]
#             coordinates_lon=all_data['coordinates'][1]
            if all_data['coordinates']==True:
                coordinates_lat=all_data['coordinates'][0]
                coordinates_lon=all_data['coordinates'][1]
            else:
                coordinates_lat=None
                coordinates_lon=None
            if all_data['place']==True:
                place_country=all_data['place_country']
                place_country_code=all_data['place_country_code']
                place_full_name=all_data['place_full_name']
                place_id=all_data['place_id']
                place_type=all_data['place_type']
            else:
                place_country=None
                place_country_code=None
                place_full_name=None
                place_id=None
                place_type=None
                
            retweeted_status=None
            
            if all_data['is_quote_status']==True:
                quoted_status_id=all_data['quoted_status_id']
                quoted_status=all_data['quoted_status']
#                 retweeted_status=all_data['retweeted_status']
                quote_count=0
            else:
                quoted_status_id=None
                quote_count=0
                quoted_status=None
#                 retweeted_status=None
            try:
                reply_count=all_data['reply_count']
            except:
                reply_count=0

            retweet_count=all_data['retweet_count']
            favorite_count=all_data['favorite_count']
            retweeted_txt=all_data['retweeted']
            entities=str(all_data['entities'])
            retweeted=0
            if retweeted_txt==True:
                retweeted=1
#             feeds_link=all_data['https://twitter.com/_/status/'+id]
            feeds_link=None
            lang=all_data['lang']
            feeds_image=None
            feeds_video=None
            # clean text
            text_preprocess = lambda x: re.compile('\#').sub('', re.compile('RT @').sub('@', x).strip())
            clean_text=text_preprocess(text)
            clean_text=re.sub('@[^\s]+','',clean_text)
            clean_text=re.sub(r"http\S+", "", clean_text)
            # end clean text
            text_sentiment_polarity=TextBlob(clean_text).sentiment[0]
            text_sentiment_subjectivity=TextBlob(clean_text).sentiment[1]


            cur.execute("INSERT INTO tweet (search_val,  created_at, created_at_date,created_at_hour,created_at_min, tweet_id,  text, source,in_reply_to_status_id, in_reply_to_user_id,  in_reply_to_screen_name,  user_name,  user_screen_name,  user_location, user_url,  user_description, user_verified, user_followers_count,  user_friends_count, user_listed_count,  user_favourites_count,  user_created_at,user_created_at_date, user_id,  user_profile_image_url_https,  coordinates_lat,  coordinates_lon, place_country,  place_country_code,  place_full_name, place_id,  place_type,  quoted_status_id,  quoted_status, retweeted_status,  quote_count,  reply_count,  retweet_count,  favorite_count, retweeted,  entities,  lang, feeds_link,  feeds_video, feeds_image,clean_text,text_sentiment_polarity,text_sentiment_subjectivity) VALUES (%s, %s,%s, %s,%s, %s,%s,%s,%s, %s, %s, %s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)", 
                        (search_val,created_at,created_at_date,created_at_hour,created_at_min,tweet_id,text,source,in_reply_to_status_id,in_reply_to_user_id,in_reply_to_screen_name,username,user_screen_name,user_location,user_url,user_description,user_verified,user_followers_count,user_friends_count,user_listed_count,user_favourites_count,user_created_at,user_created_at_date,user_id,user_profile_image_url_https,coordinates_lat,coordinates_lon,place_country,place_country_code,place_full_name,place_id,place_type,quoted_status_id,quoted_status,retweeted_status,quote_count,reply_count,retweet_count,favorite_count,retweeted,entities,lang,feeds_link,feeds_video,feeds_image,clean_text,text_sentiment_polarity,text_sentiment_subjectivity))

            connection.commit()
            print("Fetched tweets ...........\n")
            print(created_at,user_created_at)
#             tweet_count +=1

            return True

# Catch all the errors from Database or Twiteer
        except tweepy.error.TweepError:
            print("Error fetching the data")
            
        except UnicodeEncodeError:
            pass
        
        except MySQLdb.DataError as e:
            print("DataError")
            print(e)

        except MySQLdb.InternalError as e:
            print("InternalError")
            print(e)

        except MySQLdb.IntegrityError as e:
            print("IntegrityError")
            print(e)

        except MySQLdb.OperationalError as e:
            print("OperationalError")
            print(e)

        except MySQLdb.NotSupportedError as e:
            print("NotSupportedError")
            print(e)

        except MySQLdb.ProgrammingError as e:
            print("ProgrammingError")
            print(e)
        except:
            print ("Unexpected general error: ", sys.exc_info()[0])
            raise
        finally:
            print("  ")
            #             cur.close()
            #             connection.close()
            

    def on_error(self, status):
        print (status)

In [ ]:
twitterStream = Stream(auth, listener())
twitterStream.filter(track=['covid'],languages = ["en"], stall_warnings = True)

Fetched tweets ...........

2020-12-26 15:01:31 2009-08-16 00:20:54
  
Fetched tweets ...........

2020-12-26 15:01:31 2016-02-11 11:46:16
  
Fetched tweets ...........

2020-12-26 15:01:31 2016-10-18 17:57:14
  
Fetched tweets ...........

2020-12-26 15:01:31 2009-03-26 17:56:43
  
Fetched tweets ...........

2020-12-26 15:01:31 2019-04-25 08:09:33
  
  
Fetched tweets ...........

2020-12-26 15:01:32 2017-03-15 21:26:21
  
  
  
  
Fetched tweets ...........

2020-12-26 15:01:32 2017-12-01 23:46:06
  
  
Fetched tweets ...........

2020-12-26 15:01:32 2020-12-26 14:57:27
  
  
Fetched tweets ...........

2020-12-26 15:01:32 2017-02-01 00:20:49
  
Fetched tweets ...........

2020-12-26 15:01:32 2016-09-27 23:48:20
  
  
Fetched tweets ...........

2020-12-26 15:01:32 2009-07-09 06:36:54
  
Fetched tweets ...........

2020-12-26 15:01:32 2017-08-04 00:42:08
  
Fetched tweets ...........

2020-12-26 15:01:32 2017-08-15 16:25:35
  
Fetched tweets ...........

2020-12-26 15:01:32 2020-08-

  
  
  
Fetched tweets ...........

2020-12-26 15:01:46 2012-05-14 17:52:03
  
  
Fetched tweets ...........

2020-12-26 15:01:46 2010-12-21 13:53:16
  
  
  
  
  
  
Fetched tweets ...........

2020-12-26 15:01:46 2018-05-11 11:41:45
  
  
Fetched tweets ...........

2020-12-26 15:01:46 2012-07-18 20:37:50
  
Fetched tweets ...........

2020-12-26 15:01:46 2018-01-21 19:47:20
  
Fetched tweets ...........

2020-12-26 15:01:46 2019-07-04 17:21:19
  
Fetched tweets ...........

2020-12-26 15:01:46 2008-12-10 20:23:57
  
  
  
Fetched tweets ...........

2020-12-26 15:01:47 2011-06-03 10:02:06
  
  
Fetched tweets ...........

2020-12-26 15:01:47 2017-07-29 19:40:31
  
  
  
Fetched tweets ...........

2020-12-26 15:01:47 2012-01-09 23:23:50
  
Fetched tweets ...........

2020-12-26 15:01:47 2014-07-23 08:21:51
  
Fetched tweets ...........

2020-12-26 15:01:47 2013-02-08 20:56:47
  
  
Fetched tweets ...........

2020-12-26 15:01:47 2018-05-13 01:35:17
  
Fetched tweets ...........

2